# 烟幕干扰弹投放策略优化 - 问题概述与建模

## 📋 项目概述

本项目旨在解决烟幕干扰弹投放策略优化问题，通过数学建模和优化算法，为无人机烟幕干扰弹的投放提供最优策略。

### 🎯 核心问题
- **目标**: 最大化烟幕云团对真目标的遮蔽时长
- **约束**: 无人机飞行能力、干扰弹物理特性、时序限制
- **场景**: 从单一场景到多无人机多导弹的复杂协调

### 📊 问题层次
1. **问题1**: 固定策略下的遮蔽时长计算
2. **问题2**: 单枚干扰弹的最优投放策略
3. **问题3**: 单无人机多干扰弹协调策略
4. **问题4**: 多无人机单干扰弹协调策略
5. **问题5**: 多无人机多干扰弹对多导弹综合策略

In [ ]:
# 导入必要的库
import sys
import os
sys.path.append('../modules')

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown

# 导入自定义模块
from physics_core import *
from visualization import *

# 设置显示
setup_notebook_display()
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ 环境设置完成！")

## 🏗️ 数学建模框架

### 坐标系统
- **地面坐标系**: 以真目标为原点的右手直角坐标系
- **X轴**: 东向为正
- **Y轴**: 北向为正  
- **Z轴**: 天向为正

### 核心物理模型

In [ ]:
# 创建物理配置对象
config = PhysicsConfig()

# 显示物理参数
print("🔬 物理参数配置:")
print("=" * 40)
for key, value in config.to_dict().items():
    print(f"{key:<25}: {value}")

print("\n📐 坐标系定义:")
print("=" * 40)
print(f"{'真目标中心':<25}: (0, 200, 5) 米")
print(f"{'假目标中心':<25}: (0, 0, 0) 米")
print(f"{'导弹M1初位置':<25}: (20000, 0, 2000) 米")
print(f"{'无人机FY1初位置':<25}: (17800, 0, 1800) 米")

### 轨迹计算模型

#### 1. 导弹轨迹 (匀速直线运动)
$$\vec{P}_M(t) = \vec{P}_{M0} + \vec{v}_M \cdot t$$

其中：
- $\vec{P}_{M0}$: 导弹初始位置
- $\vec{v}_M$: 导弹速度矢量（指向假目标）
- $t$: 时间

#### 2. 无人机轨迹 (等高匀速直线运动)
$$\vec{P}_{FY}(t) = \vec{P}_{FY0} + \vec{v}_{FY} \cdot t$$

其中 $\vec{v}_{FY} = (v \cos\theta, v \sin\theta, 0)$

#### 3. 烟幕弹轨迹 (平抛运动)
$$\vec{P}_G(t) = \vec{P}_{drop} + \vec{v}_{drop} \cdot t + \frac{1}{2}\vec{g} \cdot t^2$$

#### 4. 烟幕云团运动 (匀速下沉)
$$\vec{P}_C(t) = \vec{P}_{det} + \vec{v}_{sink} \cdot t$$

In [ ]:
# 演示轨迹计算
print("🚀 轨迹计算演示")
print("=" * 50)

# 时间数组
time_array = np.linspace(0, 30, 100)

# 导弹轨迹
missile_initial = np.array([20000, 0, 2000])
missile_target = np.array([0, 0, 0])  # 假目标
missile_trajectory = EnhancedTrajectoryCalculator.missile_trajectory_array(
    missile_initial, missile_target, 300, time_array)

# 无人机轨迹
drone_initial = np.array([17800, 0, 1800])
drone_velocity = np.array([-120, 0, 0])  # 向西飞行
drone_trajectory = EnhancedTrajectoryCalculator.drone_trajectory_array(
    drone_initial, drone_velocity, time_array)

print(f"✅ 导弹30秒后位置: {missile_trajectory[-1]}")
print(f"✅ 无人机30秒后位置: {drone_trajectory[-1]}")
print(f"✅ 导弹速度: {np.linalg.norm(missile_trajectory[1] - missile_trajectory[0]) / (time_array[1] - time_array[0]):.1f} m/s")

### 🎯 遮蔽判断模型

核心问题：判断从导弹位置到真目标的视线是否被烟幕云团遮挡

#### 几何判据
设导弹位置为 $M$，真目标点为 $T$，烟幕球心为 $C$，半径为 $R$。

视线 $MT$ 被球遮蔽的条件：
1. **距离条件**: 球心到视线的距离 $d \leq R$
2. **位置条件**: 球心在导弹和目标之间

#### 数学表达
$$d = \frac{||\vec{MC} \times \vec{MT}||}{||\vec{MT}||} \leq R$$

$$0 \leq \vec{MC} \cdot \vec{MT} \leq ||\vec{MT}||^2$$

In [ ]:
# 遮蔽判断演示
print("🎯 遮蔽判断演示")
print("=" * 50)

# 测试场景
missile_pos = np.array([10000, 0, 1000])
target_points = [np.array([0, 200, 5])]
cloud_center = np.array([5000, 100, 500])
cloud_radius = 10.0

# 批量测试不同的云团位置
test_positions = [
    np.array([5000, 100, 500]),   # 可能遮蔽
    np.array([5000, 0, 500]),     # 可能遮蔽
    np.array([5000, 1000, 500]),  # 不太可能遮蔽
    np.array([15000, 100, 500]),  # 在导弹后方，不遮蔽
]

for i, cloud_pos in enumerate(test_positions):
    is_obscured = EnhancedObscurationAnalyzer.batch_line_sphere_intersection(
        missile_pos[np.newaxis, :], 
        target_points[0][np.newaxis, :], 
        cloud_pos, 
        cloud_radius
    )[0]
    
    status = "✅ 遮蔽" if is_obscured else "❌ 未遮蔽"
    print(f"测试 {i+1}: 云团位置 {cloud_pos} - {status}")

print("\n📊 遮蔽判断算法验证完成")

## 🎮 交互式场景演示

让我们创建一个简单的场景来演示基本的物理建模：

In [ ]:
# 创建默认场景
scenario = create_default_scenario()
print("📦 默认场景已创建")

# 创建仿真引擎
sim_engine = InteractiveSimulationEngine(time_step=0.1)

# 运行仿真
print("🚀 正在运行仿真...")
results = sim_engine.simulate_scenario(scenario, max_time=40.0)

# 显示结果摘要
print("\n📊 仿真结果摘要:")
summary = sim_engine.get_simulation_summary()
display(summary)

print(f"\n✅ 仿真完成！总计算点数: {len(results['time_array'])}")

## 📈 3D可视化展示

使用Plotly创建交互式3D可视化：

In [ ]:
# 创建可视化器
visualizer = InteractiveVisualizer()

# 生成3D可视化
fig_3d = visualizer.plot_3d_scenario_plotly(results, "基础场景 - 3D轨迹可视化")
fig_3d.show()

print("🎨 3D可视化已生成 - 可以拖拽旋转、缩放查看")

## 📊 时间线分析

显示目标遮蔽的时间线：

In [ ]:
# 生成时间线图
fig_timeline = visualizer.plot_obscuration_timeline(results)
fig_timeline.show()

print("📈 遮蔽时间线已生成")
print("💡 灰色区域表示烟幕云团活跃期，蓝色区域表示实际遮蔽时段")

## 🔧 核心算法验证

验证我们的核心计算是否正确：

In [ ]:
print("🔍 算法验证报告")
print("=" * 60)

# 验证1: 轨迹计算
t_test = 10.0
missile_pos_t = EnhancedTrajectoryCalculator.missile_trajectory_array(
    np.array([1000, 0, 100]), np.array([0, 0, 0]), 100, np.array([t_test]))[0]
expected_pos = np.array([1000, 0, 100]) + np.array([-100, 0, -10]) * t_test

error = np.linalg.norm(missile_pos_t - expected_pos)
print(f"✅ 轨迹计算验证: 误差 = {error:.6f} 米 (应接近0)")

# 验证2: 时间计算
total_time_points = len(results['time_array'])
expected_points = int(40.0 / 0.1) + 1
print(f"✅ 时间数组验证: {total_time_points} 个点 (期望 {expected_points} 个点)")

# 验证3: 物理常数
gravity_magnitude = np.linalg.norm(config.GRAVITY)
print(f"✅ 重力加速度验证: {gravity_magnitude:.1f} m/s² (标准值 9.8)")

# 验证4: 遮蔽计算
obscuration_total = sum(result['total_duration'] for result in results['obscuration_results'].values())
print(f"✅ 遮蔽计算验证: 总遮蔽时长 = {obscuration_total:.3f} 秒")

print("\n🎯 所有验证完成！算法工作正常。")

## 📝 建模总结

### ✅ 已完成的建模工作

1. **物理模型建立**
   - 轨迹计算模型（导弹、无人机、烟幕弹）
   - 遮蔽判断几何算法
   - 时间离散化仿真框架

2. **计算引擎实现**
   - 向量化高效计算
   - 批量遮蔽判断
   - 交互式仿真支持

3. **可视化系统**
   - 3D交互式轨迹显示
   - 时间线分析图表
   - 参数调节界面

### 🚀 下一步工作

- **Notebook 02**: 深入物理计算核心
- **Notebook 03**: 问题1固定策略分析
- **Notebook 04-07**: 各个优化问题求解
- **Notebook 08**: 综合结果分析

### 💡 关键洞察

1. **计算效率**: 向量化计算显著提升性能
2. **可视化价值**: 3D可视化帮助理解复杂的空间关系
3. **模型验证**: 算法验证确保计算的正确性
4. **交互性**: Jupyter环境便于参数调节和实验

---

**🎉 恭喜！** 您已经完成了烟幕干扰策略优化的基础建模工作。现在可以继续探索后续的优化问题了！

In [ ]:
# 保存当前工作环境
print("💾 保存工作环境...")

# 保存配置到文件
config_data = {
    'physics_config': config.to_dict(),
    'default_scenario': scenario,
    'simulation_results': {
        'total_time_points': len(results['time_array']),
        'obscuration_summary': sim_engine.get_simulation_summary().to_dict()
    }
}

import json
with open('../data/session_01_config.json', 'w') as f:
    # Convert numpy arrays to lists for JSON serialization
    def convert_numpy(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert_numpy(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(item) for item in obj]
        return obj
    
    json.dump(convert_numpy(config_data), f, indent=2)

print("✅ 环境已保存到 ../data/session_01_config.json")
print("\n🎯 准备就绪！可以继续下一个Notebook的学习。")